# Progress file for NER training

## [Installatie](#Installatie)

### Installeer spacy in jupyter notebook env

In [1]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install unicode

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Installeer spacy voor windows

1. Installeer anaconda
    <div>Dit geeft een goede python omgeving om in te werken.</div>


2. Installeer spacy in de anaconda terminal
<div class="alert alert-block alert-info">conda install -c conda-forge spacy </div>

### Installeer spacy for linux

In een terminal met python geinstalleerd
<div class="alert alert-block alert-info">pip install spacy && pip install unicode</div>

<div class="alert alert-block alert-warning">Nadat u de installatie volbracht heeft kan u kiezen of u het europese of het amerikaanse model wilt gebruiken. De commandos blijven hetzelfde, deze moeten enkel in de juiste map uitgevoerd worden, EU of USA.</div>

## [Extracting_data_pickle_naar_csv](#Extracting_data_pickle_naar_csv)

Dit script doet hetvolgende:
1. Dit script start met het kiezen van een pickle file.
2. Colommen **address_1, address_2, person_ctry_code** apart nemen
3. Drop all NAN records
4. Sorteren op **person_ctry_code** en enkel EU landen
5. **address_1 en address_2** in een csv file extracten

<div class="alert alert-block alert-warning">Het gedeelte in commentaar doet het zelfde maar dan met de colommen <b>street, city, zip_code.</b> Hiervoor hebben we ondervonden dat er minder ingevulde records zijn.</div>
Windows
<div class="alert alert-block alert-info">python generate-spacy-format.py</div>
Linux
<div class="alert alert-block alert-info">python3 generate-spacy-format.py</div>


### EU

In [2]:
import pickle
import pandas as pd

df = pd.read_pickle("./EU/data/samples/500ksample.pkl")

# extract_street = df.filter(items=['street', 'city', 'zip_code', 'person_ctry_code'])
# dropstreet = extract_street.dropna()
# filterstreet = dropstreet.sort_values(by=[('person_ctry_code')])
# nostreet = filterstreet[filterstreet['person_ctry_code'].isin(['AT','BE','BG','CY','CZ','DE','DK','EE','ES','FI','FR','GR','HR','HU','IE','IT','LT','LU','LV','MT','NL','PO','PT','RO','SE','SI','SK'])]
# dropstreet = dropstreet.sort_values(by=[('person_ctry_code')])
# newstreet = dropstreet[dropstreet['person_ctry_code'].isin(['AT','BE','BG','CY','CZ','DE','DK','EE','ES','FI','FR','GR','HR','HU','IE','IT','LT','LU','LV','MT','NL','PO','PT','RO','SE','SI','SK'])]]
# pf = pf.groupby(['person_ctry_code'])

pf = df.filter(items=['address_1', 'address_2', 'person_ctry_code'])
pf = pf.dropna()
pf = pf.sort_values(by=[('person_ctry_code')])
pf = pf[pf['person_ctry_code'].isin(['AT','BE','BG','CY','CZ','DE','DK','EE','ES','FI','FR','GR','HR','HU','IE','IT','LT','LU','LV','MT','NL','PO','PT','RO','SE','SI','SK'])]
result = pf.filter(items=['address_1', 'address_2','person_ctry_code'])

result.to_csv('./EU/data/samples/500ksample-europefilter-address.csv', index=False)

### USA

In [3]:
import pickle
import pandas as pd

df = pd.read_pickle("./USA/data/samples/500ksample.pkl")

#STREET CITY ZIPCODE
# extract_street = df.filter(items=['street', 'city', 'zip_code', 'person_ctry_code'])
# dropstreet = extract_street.dropna()
# filterstreet = dropstreet.sort_values(by=[('person_ctry_code')])
# nostreet = filterstreet[filterstreet['person_ctry_code'].isin(['US'])]
# dropstreet = dropstreet.sort_values(by=[('person_ctry_code')])
# newstreet = dropstreet[dropstreet['person_ctry_code'].isin(['US'])]]
# pf = pf.groupby(['person_ctry_code'])

#ADDRESS1 ADDRESS2
pf = df.filter(items=['address_1', 'address_2', 'person_ctry_code'])
pf = pf.dropna()
pf = pf.sort_values(by=[('person_ctry_code')])
pf = pf[pf['person_ctry_code'].isin(['US'])]
result = pf.filter(items=['address_1', 'address_2'])




result.to_csv('./USA/data/samples/500ksample-americanfilter-address.csv', index=False)

## [Genereer_trainingdata](#Genereer_trainingdata)

## Parsing a csv file with USAropean addresses to spaCy readable format
Dit script gaat ons trainings data bezorgen van een op voorhand gegenereerde csv file.
Doormiddel van gebruik van regex
<!-- > TODO: regex uitleggen -->
Om onze trainingsdata te genereren moeten we eerst en vooral zorgen dat onze data uit de pickel file wordt ingelezen en geparset naar het correcte formaat. In ons script ```parser.py``` checken we elke entry op verschillende patronen om te bepalen of ze bruikbaar zijn.
Eerst en vooral checken we met 
> ```(.*),(.*)``` 

of de string een komma bevat met ervoor en erna telkens een groep van karakters. Als de string voldoet aan deze voorwaarde splitsen we de string op in 2 groepen, groep 1 zijnde het straat-gedeelte van het adres en groep 2 zijnde het gemeente gedeelte van het adres. Vervolgens gaan we een aantal meer specifieke reguliere expressies toepassen om te verifieren of de string hier aan voldoet. 
Die eerste reguliere expressie die we gebruiken is 
> ```(^(.*) ([0-9]{3,5}-[0-9]{4}))```

We passen deze toe op het gemeente-gedeelte van de originele string.
Deze regex doet het volgende:
* Is er een opeenvolging van eender welke karakters (groep 1)
* volgt na deze groep 1 een spatie
* volgt na de spatie een nieuwe groep (groep 2) bestaande uit: 3, 4 of 5 cijfers 
gevolgd door een '-' gevolgd door opnieuw 4 cijfers.
Als deze groepen aanwezig zijn hebben we dus een string met de stad (groep 1) en een string met de postcode (groep 2).

Als volgende hebben we een reguliere expressie die specifiek voor Nederland geldt:
> ```(^([0-9]{4} [A-Z]{2}) (.*))```

Deze regex matcht 2 groepen:
groep 1:
* 4 opeenvolgende cijfers
* een spatie
* 2 opeenvolgende hoofdletters
groep 2:
* een ongelimiteerde opeenvolging van karakters

Als laatste hebben we 2 reguliere expressies specifiek voor de VS, Belgie en Duitsland in 2 formaten.
Formaat 1:
> ```((^[0-9]{4,5}) (.*))```

* 4 of 5 opeenvolgende cijfers
* een spatie
* een ongelimiteerde opeenvolging van karakters
Formaat 2:
> ```(^(.*) [0-9]{4,5})```

* een ongelimiteerde opeenvolging van karakters
* een spatie
* 4 of 5 opeenvolgende cijfers

<div>
Dit script maakt een "training_EU.py" file in de map /data/trainingData
</div>
<div>
De data komt in het formaar:
</div>

<blockquote>
TRAININGS_DATA = [<br>
    &emsp;("Heipelweg 7,A-8700 Leoben",[(0,9, "STREET"),(10,11, "NUMBER"),(19,25, "CITY"),(12,18, "ZIPCODE")]),<br>
    &emsp;...<br>
    ]<br>
</blockquote>

#### Windows
<div class="alert alert-block alert-info">python generate-spacy-format.py</div>

#### Linux
<div class="alert alert-block alert-info">python3 generate-spacy-format.py</div>



### EU

In [4]:
import csv
from itertools import count
import re
from 0 import unidecode

with open('./EU/data/samples/500ksample-europefilter-address.csv', newline="\n") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    counter = 0
    f = open("./EU/output/fouten_generate.txt", 'w')
    f.close
    f = open("./EU/data/validation/validation_EU.txt", 'w')
    f.close
    f = open("./EU/data/trainingData/training_EU.py", 'w')
    f.write("TRAININGS_DATA = [\n")
    f.close()
    f = open("./EU/data/trainingData/dev_EU.py", 'w')
    f.write("TRAININGS_DATA = [\n")
    f.close()
    fail_numberstreet = 0
    fail_cityzip = 0 
    for row in csvreader:
        correct = True
        adres = row[0] + ',' + row[1]
        adres = unidecode(adres)
        if not re.match(".*,.*,.*", adres):
            adreslengte = len(adres)
            adres1 = row[0]
            totale_lengte_straat = len(adres1) + 1
            adres2 = row[1]
            # nummer + straat
            if re.match('^([0-9\-]+( ?bis)?) ([ \u00C0-\u017Fa-zA-Z\'\-]+)', adres1):
                split = re.search('^([0-9\-]+( ?bis)?) ([ \u00C0-\u017Fa-zA-Z\'\-]+)', adres1)
                straat = split.span(3)
                nummer = split.span(1)

            # straat + nummer
            elif re.match('^([ \u00C0-\u017Fa-zA-Z\'\-]+) ([0-9\/\-]+( ?[a-zA-Z]*)?)', adres1):
                split = re.search('^([ \u00C0-\u017Fa-zA-Z\'\-]+) ([0-9\/\-]+( ?[a-zA-Z]*)?)', adres1)
                straat = split.span(1)
                nummer = split.span(2)
            else:
                f = open("./EU/output/fouten_generate.txt", 'a')
                f.write("street: " + adres + "\n")
                f.close()
                fail_numberstreet +=1
                correct = False
            
            # zipcode + city
            if re.match('([ A-Z-]*[0-9]+( ?[A-Z]{2})?) ?([\u00C0-\u0338a-zA-Z ./-]+)$', adres2):
                split = re.search('([ A-Z-]*[0-9]+( ?[A-Z]{2})?) ?([\u00C0-\u0338a-zA-Z ./-]+)$', adres2)
                zipcode = list(split.span(1))
                city = list(split.span(3))
                zipcode[0] += totale_lengte_straat
                zipcode[1] += totale_lengte_straat
                city[0] += totale_lengte_straat
                city[1] += totale_lengte_straat

            # city + zipcode 
            elif re.match('([\u00C0-\u017Fa-zA-Z ./-]+) ?([ A-Z-]*[0-9]+)$', adres2):
                split = re.search('([\u00C0-\u017Fa-zA-Z ./-]+) ?([ A-Z-]*[0-9]+)$', adres2)
                zipcode = list(split.span(2))
                city = list(split.span(1))
                zipcode[0] += totale_lengte_straat
                zipcode[1] += totale_lengte_straat
                city[0] += totale_lengte_straat
                city[1] += totale_lengte_straat
            else:
                f = open("./EU/output/fouten_generate.txt", 'a')
                f.write("city: " + adres + "\n")
                f.close()
                fail_cityzip += 1
                correct = False

            if correct:
                counter += 1
                if counter < 5:
                    f = open("./EU/data/trainingData/training_EU.py", 'a')
                    f.write("\t(\""+adres+"\","+"[("+str(straat[0])+","+str(straat[1])+", \"STREET\"),("+str(nummer[0])+","+str(nummer[1])+", \"NUMBER\"),("+str(city[0])+","+str(city[1])+", \"CITY\"),("+str(zipcode[0])+","+str(zipcode[1])+", \"ZIPCODE\")]),\n")
                    f.close()
                else:
                    f = open("./EU/data/trainingData/dev_EU.py", 'a')
                    f.write("\t(\""+adres+"\","+"[("+str(straat[0])+","+str(straat[1])+", \"STREET\"),("+str(nummer[0])+","+str(nummer[1])+", \"NUMBER\"),("+str(city[0])+","+str(city[1])+", \"CITY\"),("+str(zipcode[0])+","+str(zipcode[1])+", \"ZIPCODE\")]),\n")
                    f.close()
                    counter = 0
                f = open("./EU/data/validation/validation_EU.txt", 'a')
                f.write(adres+"&"+str(straat[0])+";"+str(straat[1])+";STREET&"+str(nummer[0])+";"+str(nummer[1])+";NUMBER&"+str(city[0])+";"+str(city[1])+";CITY&"+str(zipcode[0])+";"+str(zipcode[1])+";ZIPCODE\n")
                f.close()
            # print("(\""+adres+"\","+"[("+str(straatposbegin)+","+str(straatposeind)+", \"STREET\"),("+str(nummerposbegin)+","+str(nummerposeind)+", \"NUMBER\"),("+str(cityposbegin)+","+str(cityposeind)+", \"CITY\"),("+str(zipcodeposbegin)+","+str(zipcodeposeind)+", \"ZIPCODE\")])")
    f = open("./EU/data/trainingData/training_EU.py", "a")
    f.write("]")
    f.close()
    f = open("./EU/data/trainingData/dev_EU.py", "a")
    f.write("]")
    f.close()
    print("failed on street: ", fail_numberstreet)
    print("failed on city: ", fail_cityzip)

failed on street:  3060
failed on city:  1935


### USA

In [5]:
import csv
from itertools import count
import re
from unidecode import unidecode
def fout(element,adres, fails):
    f = open("./USA/output/fouten_generate.txt", 'a')
    f.write(element + ": " + adres + "\n")
    f.close()
    return fails + 1

with open('./USA/data/samples/500ksample-americanfilter-address.csv', newline="\n") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    counter = 0 
    f = open("./USA/output/fouten_generate.txt", 'w')
    f.close
    f = open("./USA/data/trainingData/training_USA.py", 'w')
    f.write("TRAININGS_DATA = [\n")
    f.close()
    f = open("./USA/data/trainingData/dev_USA.py", 'w')
    f.write("TRAININGS_DATA = [\n")
    f.close()
    f = open("./USA/data/validation/validation_USA.txt", 'w')
    f.close
    fail_nummer = 0
    fail_straat = 0
    fail_city = 0
    fail_zipcode = 0
    fail_state = 0
    counter = 0
    nummmer = ""
    straat = ""
    city = ""
    zipcode = ""
    state = ""
    for row in csvreader:
        correct = False
        adres = ','.join(row)
        adres = unidecode(adres)
        adres1 = row[0]
        adres2 = row[1]
        totale_lengte_straat = len(adres1) + 1
        full_string = "\t(\""+adres+"\","+"["
        #nummer
        if re.match('(/ )?([0-9]+)', adres1):
            nummer = re.search('(/ )?([0-9]+)', adres1).span(2)
            full_string += "("+str(nummer[0])+","+str(nummer[1])+", \"NUMBER\"),"
            correct = True
        else:
            fail_nummer = fout("Nummer", adres1, fail_nummer)
            continue
        #straat
        if re.search('[0-9]+ ([\s\S]*)$', adres1):
            straat = re.search('[0-9]+ ([\s\S]*)$', adres1).span(1)
            full_string += "("+str(straat[0])+","+str(straat[1])+", \"STREET\"),"
            correct = True
        else:
            fail_straat = fout("Straat", adres1, fail_straat)
            continue
        # if not re.match('([0-9]+)', adres1) and not re.match('[0-9]+ ([\s\S]*)$', adres1):
            
        #city
        if re.match('([a-zA-Z ]+?(?=,))', adres2):
            city = re.search('([a-zA-Z ]+?(?=,))', adres2).span(1)
            city = list(city)
            city[0] += totale_lengte_straat
            city[1] += totale_lengte_straat
            full_string += "("+str(city[0])+","+str(city[1])+", \"CITY\"),"
            correct = True
        else:
            fail_city = fout("City", adres2, fail_city)
            continue
        #zipcode
        if re.search('([0-9\-]+$)', adres2):
            zipcode = re.search('([0-9\-]+$)', adres2).span(1)
            zipcode = list(zipcode)
            zipcode[0] += totale_lengte_straat
            zipcode[1] += totale_lengte_straat
            full_string += "("+str(zipcode[0])+","+str(zipcode[1])+", \"ZIPCODE\"),"
            correct = True
        else:
            fail_zipcode = fout("Zipcode", adres2, fail_zipcode)
            continue
        #state
        if re.search('(,? )([ A-Za-z]*)(?<! ),? [0-9]', adres2):
            state = re.search('(,? )([ A-Za-z]*)(?<! ),? [0-9]', adres2).span(2)
            state = list(state)
            state[0] += totale_lengte_straat
            state[1] += totale_lengte_straat
            if not city[1] >= state[0]:
                full_string += "("+ str(state[0])+","+str(state[1])+", \"STATE\")"
                correct = True
        else:
            fail_state = fout("State", adres2, fail_state)
            continue    

        if correct:
            counter += 1
            if counter < 5:
                f = open("./USA/data/trainingData/training_USA.py", 'a')
                f.write(full_string + "]),\n")
                f.close()
            else:
                f = open("./USA/data/trainingData/dev_USA.py", 'a')
                f.write(full_string + "]),\n")
                f.close()
                counter = 0
            f = open("./USA/data/validation/validation_USA.txt", 'a')
            f.write(adres+"&"+str(straat[0])+";"+str(straat[1])+";STREET&"+str(nummer[0])+";"+str(nummer[1])+";NUMBER&"+str(city[0])+";"+str(city[1])+";CITY&"+str(zipcode[0])+";"+str(zipcode[1])+";ZIPCODE&"+str(state[0])+";"+str(state[1])+";STATE\n")
            f.close()
            # print("(\""+adres+"\","+"[("+str(straatposbegin)+","+str(straatposeind)+", \"STREET\"),("+str(nummerposbegin)+","+str(nummerposeind)+", \"NUMBER\"),("+str(cityposbegin)+","+str(cityposeind)+", \"CITY\"),("+str(zipcodeposbegin)+","+str(zipcodeposeind)+", \"ZIPCODE\")])")
    f = open("./USA/data/trainingData/training_USA.py", "a")
    f.write("]")
    f.close()
    f = open("./USA/data/trainingData/dev_USA.py", "a")
    f.write("]")
    f.close()
    print("failed on street: ", fail_straat)
    print("failed on number: ", fail_nummer)
    print("failed on city: ", fail_city)
    print("failed on zipcode: ", fail_zipcode)
    print("failed on state: ", fail_state)

failed on street:  251
failed on number:  2376
failed on city:  2320
failed on zipcode:  68
failed on state:  87


## [Prepare_binarydata](#Prepare_binarydata)

In geval van Europa wordt de "/EU/data/trainingData/training_EU.py" file omgezet naar binaire data. <br>
Dit wordt dan in de map "/EU/data/binaryData/" als spacy file aangemaakt. De "train.spacy" is de data dat gebruikt wordt als training. Dit is 80% van de totale gegenereerde trainingsdata <br>
"dev.spacy" is de validatiedata dat gebruikt wordt bij het trainen van het model. Dit is de resterende 20% van de totale trainingsdata.

#### Windows
<div class="alert alert-block alert-info">python PrepareData.py</div>

#### Linux
<div class="alert alert-block alert-info">python3 PrepareData.py</div>

### PrepareData.py

### EU

In [6]:
import spacy
import sys
from spacy.tokens import DocBin
sys.path.insert(0, './EU/data/trainingData')
from training_EU import TRAININGS_DATA as TD

f = open("./EU/output/problems.txt", 'w')
f.close()


nlp = spacy.blank("en")

infixes = nlp.Defaults.infixes + [r'\,']
nlp.tokenizer.infix_finditer = spacy.util.compile_infix_regex(infixes).finditer

db = DocBin()
for text, annotations in TD:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is None:
            f = open("./EU/output/problems.txt", 'a')
            f.write("Problem:" + doc.text + " (" + doc.text[start:end] + ")\n")
            f.close()
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./EU/data/binaryData/train.spacy")

db = DocBin()
for text, annotations in TD:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./EU/data/binaryData/dev.spacy")

### USA

In [7]:
import spacy
import sys
from spacy.tokens import DocBin
sys.path.insert(0, './USA/data/trainingData')
from training_USA import TRAININGS_DATA as TD
from dev_USA import TRAININGS_DATA as DD

f = open("./USA/output/fouten_dataprepare.txt", 'w')
f.close()

nlp = spacy.blank("en")

infixes = nlp.Defaults.infixes + [r'\,']
nlp.tokenizer.infix_finditer = spacy.util.compile_infix_regex(infixes).finditer

db = DocBin()
for text, annotations in TD:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is None:
            f = open("./USA/output/fouten_dataprepare.txt", 'a')
            f.write("Problem:" + doc.text + " (" + doc.text[start:end] + ")\n")
            f.close()
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./USA/data/binaryData/train.spacy")

db = DocBin()
for text, annotations in DD:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./USA/data/binaryData/dev.spacy")

## [Start_training](#Start_training)

Hier komt het moment waar we kunnen beginnen met trainen van het model. Dit gebeurt via de command line. <br>
De spacy train geeft aan dat we de training starten. Vervolgens wordt de config file meegegeven. Bij het "--output" argument staat de locatie waar we het model opslaan. In deze map komt het best gevonden model en het laatste aangemaakte model.<br>
Tot slot geven we onze custom Tokenizer mee deze zorgt ervoor dat we steeds splitsen bij een comma, hiermee krijgen we de comma als een aparte token. Dit verhelpt het probleem dat het model commagetallen gaat lezen.
> Bijvoorbeeld:  "Am Kirchwald 347,6100 Seefeld" => "Am"; "Kirchwald"; "347"; ","; "6100"; "Seefeld"

#### Windows
<div class="alert alert-block alert-info">python TrainModel.py</div>

#### Linux
<div class="alert alert-block alert-info">python3 TrainModel.py</div>

### Run training

### EU

In [8]:
import os
os.system("python3 -m spacy train config_EU.cfg --output ./EU/model --code ./EU/config/CustomTokenizer.py")

[2022-03-07 16:48:54,004] [INFO] Set up nlp object from config
[2022-03-07 16:48:54,014] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-07 16:48:54,134] [INFO] Created vocabulary
[2022-03-07 16:48:54,135] [INFO] Finished initializing nlp object
[2022-03-07 16:48:58,669] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


ℹ Saving to output directory: EU/model
ℹ Using CPU

=========================== Initializing pipeline ===========================
Ik heb een rode patat
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     79.00    9.64    7.61   13.13    0.10
  0     200        159.82   4118.32   95.98   93.66   98.42    0.96
  1     400        139.52   1930.88   96.68   94.28   99.21    0.97
  2     600        168.12   2149.74   97.77   96.58   98.98    0.98
  3     800        213.61   1766.20   98.45   97.75   99.15    0.98
  4    1000        213.62   1276.51   98.82   98.54   99.10    0.99
  5    1200        220.16   1183.02   99.15   99.54   98.77    0.99
  7    1400        219.21    894.81   99.47   99.58   99.36    0.99
  9    

0

### USA

In [9]:
import os
os.system("python3 -m spacy train config_USA.cfg --output ./USA/model --code ./USA/config/CustomTokenizer.py")

[2022-03-07 16:52:48,526] [INFO] Set up nlp object from config
[2022-03-07 16:52:48,536] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-07 16:52:48,660] [INFO] Created vocabulary
[2022-03-07 16:52:48,661] [INFO] Finished initializing nlp object
[2022-03-07 16:52:53,409] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


ℹ Saving to output directory: USA/model
ℹ Using CPU

=========================== Initializing pipeline ===========================
Ik heb een rode patat
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     72.50   13.97   10.74   19.99    0.14
  0     200        119.18   2649.55   99.25   99.06   99.43    0.99
  0     400        128.43    281.56   99.40   99.23   99.57    0.99
  1     600        135.12    194.39   99.57   99.48   99.66    1.00
  2     800        266.25    227.30   99.61   99.50   99.72    1.00
  2    1000        205.38    156.96   99.65   99.56   99.74    1.00
  3    1200        311.72    176.16   99.52   99.41   99.63    1.00
  5    1400        290.19    129.04   99.58   99.47   99.70    1.00
  6   

0

## [Recall_And_Precision_plus_Visualiseer_model](#Recall_And_Precision_plus_Visualiseer_model)

In volgende code kan u een visueel beeld krijgen van uitkomsten die het model heeft gevonden.<br>
Dit kan gedaan worden door een csv file mee te geven. Of door zelf adressen in te geven in de list die momenteel in commentaar staat. <br>
Erna wordt het model gekozen hier hebt u de keuze uit het beste model of het laatste model.
<div>nlp = spacy.load("./model/model-best") of<br>
nlp = spacy.load("./model/model-last")</div>


#### Windows
<div class="alert alert-block alert-info">python TestModel.py</div>

#### Linux
<div class="alert alert-block alert-info">python3 TestModel.py</div>

<div class="alert alert-block alert-warning">We hebben de laatste 3 regels in commentaar geplaatst zodat de output van de Jupyter notebook niet te lang werd. We raden aan om het commando in terminal uit te voeren als u de volledige uitkomst wilt bekijken. U kan het resultaat bekijken door naar de link <a href="http://localhost:5000">http://localhost:5000</a> te gaan.</div>

### EU

In [10]:
import csv
from curses.ascii import ctrl
import spacy
from spacy.tokens import DocBin
from spacy import displacy

# test_text = ["Keilalahdentie 4,02150 Espoo", "Weisshsusstrasse 2,52066 Aachen", "14 rue Royale,75008 Paris", "Hüninger Strasse 25,14195 Berlin", "19 bis rue Hoche,49100 Angers"]
test_text = []
with open('./EU/data/samples/500ksample-europefilter-address.csv', newline="\n") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        row = (row[0] + ',' + row[1],row[2])
        # print(row)
        test_text.append(row)

labeledData = open('./EU/data/validation/validation_EU.txt', 'r')
adreslist = []
for line in labeledData.readlines():
    line = line.strip('\n')
    line = line.split("&")
    record = []
    recordtuple = (line[0],record)
    for i in line:
        i = i.split(";")
        record.append(i)
    record.pop(0)
    adreslist.append(recordtuple)

nlp = spacy.load("./EU/model/model-best")
# nlp = spacy.load("./EU/model/model-last")
lijst = []
correct_counter = 0
total_counter = 0
total_counter_straat = 0
total_counter_nummer = 0
total_counter_city = 0
total_counter_zipcode = 0
ctry_code = ""
for i in test_text:
    adres_ctry_code = i[1]
    # print(adres_ctry_code)
    # print(i[0])
    doc = nlp(i[0])
    ents = list(doc.ents)
    for at in adreslist:
        if at[0] == str(doc):
            for j in at[1]:
                for token in ents:
                    # print(token)
                    bool1 = str(j[0]) == str(token.start_char)
                    bool2 = str(j[1]) == str(token.end_char)
                    bool3 = j[2] == token.label_
                    if bool1 and bool2 and bool3 and token.label_ == "STREET":
                        total_counter_straat +=1
                    if bool1 and bool2 and bool3 and token.label_ == "NUMBER":
                        total_counter_nummer +=1
                    if bool1 and bool2 and bool3 and token.label_ == "CITY":
                        total_counter_city +=1
                    if bool1 and bool2 and bool3 and token.label_ == "ZIPCODE":
                        total_counter_zipcode +=1
    lijst.append(doc)
    total_counter += 1
    if ctry_code != adres_ctry_code:
        print("----------------------" + ctry_code + "--------------------------------")
        print("straat: " + str(total_counter_straat) + " percentage: " + str(total_counter_straat / total_counter * 100)+ "%")
        print("nummer: " + str(total_counter_nummer) + " percentage: " + str(total_counter_nummer / total_counter * 100)+ "%")
        print("city: " + str(total_counter_city) + " percentage: " + str(total_counter_city / total_counter * 100)+ "%")
        print("zipcode: " + str(total_counter_zipcode) + " percentage: " + str(total_counter_zipcode / total_counter * 100)+ "%")
        print("counter: " + str(total_counter))
        ctry_code = adres_ctry_code
        total_counter = 0
        total_counter_straat = 0
        total_counter_nummer = 0
        total_counter_city = 0
        total_counter_zipcode = 0



# colors = {"STREET": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "NUMBER": "linear-gradient(90deg, #3f5efb, #fc466b)", "ZIPCODE": "linear-gradient(90deg, #090979, #00d4ff)", "CITY": "linear-gradient(90deg, #eeaeca, #94bbe9)", "OTHER": "linear-gradient(90deg, #22c1c3, #fdbb2d)",}
# options = {"ents": ["CITY", "STREET", "NUMBER", "ZIPCODE", "OTHER"], "colors": colors}
# displacy.serve(lijst, style="ent", options=options)



------------------------------------------------------
straat: 0 percentage: 0.0%
nummer: 0 percentage: 0.0%
city: 0 percentage: 0.0%
zipcode: 0 percentage: 0.0%
counter: 1
----------------------person_ctry_code--------------------------------
straat: 1 percentage: 100.0%
nummer: 1 percentage: 100.0%
city: 1 percentage: 100.0%
zipcode: 1 percentage: 100.0%
counter: 1
----------------------AT--------------------------------
straat: 264 percentage: 67.00507614213198%
nummer: 259 percentage: 65.73604060913706%
city: 260 percentage: 65.98984771573603%
zipcode: 263 percentage: 66.75126903553299%
counter: 394
----------------------BE--------------------------------
straat: 263 percentage: 55.838641188959656%
nummer: 259 percentage: 54.989384288747345%
city: 254 percentage: 53.92781316348195%
zipcode: 258 percentage: 54.77707006369427%
counter: 471
----------------------BG--------------------------------
straat: 0 percentage: 0.0%
nummer: 0 percentage: 0.0%
city: 0 percentage: 0.0%
zipcode: 0

### USA

In [11]:
import csv
import spacy
from spacy.tokens import DocBin
from spacy import displacy

# test_text = ["Keilalahdentie 4,02150 Espoo", "Weisshsusstrasse 2,52066 Aachen", "14 rue Royale,75008 Paris", "Hüninger Strasse 25,14195 Berlin", "19 bis rue Hoche,49100 Angers"]
test_text = []
with open('./USA/data/samples/500ksample-americanfilter-address.csv', newline="\n") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        row = ','.join(row)
        test_text.append(row)

labeledData = open('./USA/data/validation/validation_USA.txt', 'r')
adreslist = []
for line in labeledData.readlines():
    line = line.strip('\n')
    line = line.split("&")
    record = []
    recordtuple = (line[0],record)
    for i in line:
        i = i.split(";")
        record.append(i)
        # print(record)
    record.pop(0)
    adreslist.append(recordtuple)

nlp = spacy.load("./USA/model/model-best")
# nlp = spacy.load("./USA/model/model-last")
lijst = []
correct_counter = 0
total_counter = 0
total_correct_straat = 0
total_correct_nummer = 0
total_correct_city = 0
total_correct_zipcode = 0
total_correct_state = 0

total_straat = 0
total_nummer = 0
total_city = 0
total_zipcode = 0
total_state = 0

for i in test_text:    
    doc = nlp(i)
    ents = list(doc.ents)
    for at in adreslist:
        if at[0] == str(doc):
            for j in at[1]:
                for token in ents:
                    # print(token)
                    bool1 = str(j[0]) == str(token.start_char)
                    bool2 = str(j[1]) == str(token.end_char)
                    bool3 = j[2] == token.label_
                    if bool1 and bool2 and bool3 and token.label_ == "STREET":
                        total_correct_straat +=1
                    if bool1 and bool2 and bool3 and token.label_ == "NUMBER":
                        total_correct_nummer +=1
                    if bool1 and bool2 and bool3 and token.label_ == "CITY":
                        total_correct_city +=1
                    if bool1 and bool2 and bool3 and token.label_ == "ZIPCODE":
                        total_correct_zipcode +=1
                    if bool1 and bool2 and bool3 and token.label_ == "STATE":
                        total_correct_state +=1
                if j[2] == "STREET":
                    total_straat += 1
                if j[2] == "NUMBER":
                    total_nummer += 1
                if j[2] == "CITY":
                    total_city += 1
                if j[2] == "ZIPCODE":
                    total_zipcode += 1
                if j[2] == "STATE":
                    total_state += 1
    lijst.append(doc)
    total_counter += 1
print("straat: " + str(total_correct_straat) + " percentage: " + str(total_correct_straat / total_straat * 100)+ "%")
print("nummer: " + str(total_correct_nummer) + " percentage: " + str(total_correct_nummer / total_nummer * 100)+ "%")
print("city: " + str(total_correct_city) + " percentage: " + str(total_correct_city / total_city * 100)+ "%")
print("zipzode: " + str(total_correct_zipcode) + " percentage: " + str(total_correct_zipcode / total_zipcode * 100)+ "%")
print("state: " + str(total_correct_state) + " percentage: " + str(total_correct_state / total_state * 100)+ "%")
print("adress in list: " + str(total_counter))

# colors = {"STREET": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "NUMBER": "linear-gradient(90deg, #3f5efb, #fc466b)", "ZIPCODE": "linear-gradient(90deg, #090979, #00d4ff)", "CITY": "linear-gradient(90deg, #eeaeca, #94bbe9)", "STATE": "linear-gradient(90deg, #22c1c3, #fdbb2d)",}
# options = {"ents": ["CITY", "STREET", "NUMBER", "ZIPCODE", "STATE"], "colors": colors}
# displacy.serve(lijst, style="ent", options=options)



straat: 9992 percentage: 99.80023971234519%
nummer: 10000 percentage: 99.88014382740711%
city: 10007 percentage: 99.9500599280863%
zipzode: 10009 percentage: 99.97003595685177%
state: 9993 percentage: 99.81022772672793%
adress in list: 13047
